# Data Preparation

In [68]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [69]:
df  = pd.read_csv('hotel_bookings.csv')

In [70]:
df.shape

(119390, 32)

In [71]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [73]:
df.shape

(119390, 32)

In [74]:
from sklearn.preprocessing import LabelEncoder

def labelencoder_fit(df,list_columns):

  results = {}

  # results
  # {'nama kolom':encoder,
  #  'kolom 2':encoder2
  #}

  for columns in list_columns:
    encoder = LabelEncoder()
    encoder.fit(df[columns])
    results[columns] = encoder

  return results

def labelencoder_transform(df,list_columns,dict_encoder):

  f_transformed_list = []

  for columns in list_columns:
    
    f_transformed = dict_encoder[columns].transform(df[columns])
    f_transformed_list.append(pd.Series(f_transformed))
  
  df_results = pd.concat(f_transformed_list,1)
  df_results.columns = list_columns
  return df_results

In [75]:
list_columns = df.select_dtypes('object').columns

dict_encoder = labelencoder_fit(df,list_columns)
df_transformed = labelencoder_transform(df,list_columns,dict_encoder)

df[list_columns] = df_transformed[list_columns]

In [76]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,1,0,342,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,3,0,NaN,NaN,0,2,0.0,0,0,1,121
1,1,0,737,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,4,0,NaN,NaN,0,2,0.0,0,0,1,121
2,1,0,7,2015,5,27,1,0,1,1,0.0,0,0,59,3,1,0,0,0,0,2,0,0,NaN,NaN,0,2,75.0,0,0,1,122
3,1,0,13,2015,5,27,1,0,1,1,0.0,0,0,59,2,0,0,0,0,0,0,0,0,304.0,NaN,0,2,75.0,0,0,1,122
4,1,0,14,2015,5,27,1,0,2,2,0.0,0,0,59,6,3,0,0,0,0,0,0,0,240.0,NaN,0,2,98.0,0,1,1,123


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  int64  
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  int64  
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [78]:
df.shape

(119390, 32)

In [79]:
df.fillna(0,inplace=True)

# Modeling

In [80]:
X = df.drop(['is_canceled'],1)
y = df['is_canceled']

In [81]:
from sklearn.model_selection import train_test_split

# Train Test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.9, stratify = y, random_state = 10)

# Labeled and Unlabeled
X_train_lab,X_test_unlab,y_train_lab,y_test_unlab = train_test_split(X_train,y_train, stratify = y_train,test_size = 0.5, random_state = 10)

In [82]:
# Summarize training Data
print('Labeled Train Set:',X_train_lab.shape,y_train_lab.shape)
print('Unlabeled Train Set:',X_test_unlab.shape,y_test_unlab.shape)

Labeled Train Set: (5969, 31) (5969,)
Unlabeled Train Set: (5970, 31) (5970,)


In [83]:
# Summarize Test Size
print('Test Set:', X_test.shape,y_test.shape)

Test Set: (107451, 31) (107451,)


## Logistic Regression

In [84]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_lab,y_train_lab)

LogisticRegression()

In [85]:
from sklearn.metrics import accuracy_score
yhat = logreg.predict(X_test)
score = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % (score * 100))

Accuracy: 72.860


## Label Propagation

In [86]:
# labeled : 0,1,2,3,4,5,...
# unlabeled : -1

X_train_mixed = np.concatenate((X_train_lab,X_test_unlab))
nolabel = [-1 for _ in range(len(y_test_unlab))]

y_train_mixed = np.concatenate((y_train_lab,nolabel))

In [87]:
X_train_mixed.shape

(11939, 31)

In [88]:
y_train_mixed.shape

(11939,)

In [96]:
from sklearn.semi_supervised import LabelPropagation

model = LabelPropagation(kernel='knn')
model.fit(X_train_mixed,y_train_mixed)

LabelPropagation(kernel='knn')

In [97]:
yhat = model.predict(X_test)
score = accuracy_score(y_test,yhat)
print('Accuracy: %.3f'%(score * 100))

Accuracy: 80.436


## Label Spreading

In [102]:
from sklearn.semi_supervised import LabelSpreading
model2 = LabelSpreading(kernel='knn',alpha=0.5)
model2.fit(X_train_mixed,y_train_mixed)


LabelSpreading(alpha=0.5, kernel='knn')

In [103]:
yhat = model2.predict(X_test)
score = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % (score * 100))

Accuracy: 80.533


## Combine Semi-supervised + Supervised Learning

In [110]:
train_labels = model2.transduction_

In [111]:
train_labels

array([1, 0, 0, ..., 0, 0, 0])

In [118]:
# label : 200
# unlabeled : 500
# real : 1,000,000

# semi-supervised = unlabeled -> label_pred
# train_labels =  label + label_pred (transduction_)
# supervised learning = train_labels
from sklearn.ensemble import RandomForestClassifier
model_transduct = RandomForestClassifier()
model_transduct.fit(X_train_mixed, train_labels)

RandomForestClassifier()

In [119]:
yhat = model_transduct.predict(X_test)
score = accuracy_score(y_test,yhat)

In [120]:
print('Accuracy: %.3f' % (score * 100))

Accuracy: 96.216


## Random Forest

In [121]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_lab,y_train_lab)

RandomForestClassifier()

In [123]:
yhat = model_rf.predict(X_test)
score = accuracy_score(y_test,yhat)

In [124]:
print('Accuracy: %.3f' % (score * 100))

Accuracy: 99.403
